In [1]:
import graphlab

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to charlesf.0@gmail.comand will expire on September 22, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-436 - Server binary: C:\Users\Home\Anaconda\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Home\AppData\Local\Temp\graphlab_server_1445385766.log.0
[INFO] GraphLab Server Version: 1.6


In [3]:
products.head(3)

Columns:
	name	str
	review	str
	rating	float

Rows: 3

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
+-------------------------------+-------------------------------+--------+
[3 rows x 3 columns]

In [4]:
# build wordcount vector
products['word_count'] = graphlab.text_analytics.count_words(products['review'])

In [5]:
products.head(3)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict

Rows: 3

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|    Planetwise Flannel Wipes   | These flannel wipes are OK... |  3.0   |
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+
|           word_count          |
+-------------------------------+
| {'and': 5L, 'stink': 1L, '... |
| {'and': 3L, 'love': 1L, 'i... |
| {'and': 2L, 'quilt': 1L, '... |
+-------------------------------+
[3 rows x 4 columns]

In [6]:
graphlab.canvas.set_target('ipynb')

[WARNING] This Python session does not appear to be running in an interactive IPython Notebook. Use of the 'ipynb' target may behave unexpectedly or result in errors.


In [7]:
products['name'].show()

In [8]:
# vulli sophie
giraffe_reviews = products[products['name']  == 'Vulli Sophie the Giraffe Teether']

In [9]:
len(giraffe_reviews)

785

In [10]:
giraffe_reviews['rating'].show(view = 'Categorical')

In [11]:
# build sentiment classifier
products['rating'].show(view = 'Categorical')

In [12]:
# define positive vs negative
products = products[products['rating'] != 3]

In [13]:
products['sentiment'] = products['rating'] >= 4

In [14]:
products.head(3)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	sentiment	int

Rows: 3

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|     Planetwise Wipe Pouch     | it came early and was not ... |  5.0   |
| Annas Dream Full Quilt wit... | Very soft and comfortable ... |  5.0   |
| Stop Pacifier Sucking with... | This is a product well wor... |  5.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|           word_count          | sentiment |
+-------------------------------+-----------+
| {'and': 3L, 'love': 1L, 'i... |     1     |
| {'and': 2L, 'quilt': 1L, '... |     1     |
| {'ingenious': 1L, 'and': 3... |     1     |
+-------------------------------+-----------+
[3 rows x 5 columns]

In [15]:
# split test and train
train_data, test_data = products.random_split(.8, seed = 0)

In [16]:
sent_mod = graphlab.logistic_classifier.create(train_data,
                                              target = 'sentiment',
                                              features = ['word_count'],
                                              validation_set = test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 219217
PROGRESS: Number of coefficients    : 219218
PROGRESS: Starting L-BFGS
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+-----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 5        | 0.000002  | 2.099014     | 0.841481          | 0.839989            |
PROGRESS: | 2         | 9        | 3.000000  | 3.171050     | 0.947425          | 0.894877            |
PROGRESS: | 3         | 10       | 3.000000  | 3.578449     | 0.92

In [95]:
sent_mod['coefficients']

Columns:
	name	str
	index	str
	class	int
	value	float

Rows: 219218

Data:
+-------------+---------------+-------+------------------+
|     name    |     index     | class |      value       |
+-------------+---------------+-------+------------------+
| (intercept) |      None     |   1   |  0.729182482603  |
|  word_count |      it.      |   1   | 0.0923459975112  |
|  word_count |   recommend   |   1   |  0.351653944839  |
|  word_count |      love     |   1   |  0.824676597257  |
|  word_count |       it      |   1   | 0.00340245508889 |
|  word_count | disappointed. |   1   |  -2.66907012284  |
|  word_count |     planet    |   1   |  -0.28318516271  |
|  word_count |      and      |   1   | 0.0387848304637  |
|  word_count |      bags     |   1   |  0.132287521499  |
|  word_count |     wipes     |   1   | -0.0146873544927 |
+-------------+---------------+-------+------------------+
[219218 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [17]:
# evalue the ROC curve
sent_mod.evaluate(test_data, metric = 'roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+----------------+------------------+-------+------+
 |    threshold     |      fpr       |       tpr        |   p   |  n   |
 +------------------+----------------+------------------+-------+------+
 |       0.0        | 0.218961625282 | 0.00575243235566 | 28162 | 5316 |
 | 0.0010000000475  | 0.781038374718 |  0.994247567644  | 28162 | 5316 |
 | 0.00200000009499 | 0.741158765989 |  0.993040267026  | 28162 | 5316 |
 | 0.00300000002608 | 0.718397291196 |  0.992401107876  | 28162 | 5316 |
 | 0.00400000018999 | 0.702972159518 |  0.991832966409  | 28162 | 5316 |
 | 0.00499999988824 | 0.691873589165 |  0.99144236915   | 28162 | 5316 |
 | 0.00600000005215 | 0.682091798345 |  0.990874227683  | 28162 | 5316 |
 | 0.00700000021607 | 0.671557562077 |  0.990483630424  | 28162 | 5316 |
 | 0.00800000037998 | 0.661399548533 |  0.99019955969   | 28162 | 5316 |
 | 0.00899999961257 

In [91]:
sent_mod.evaluate(test_data)

{'accuracy': 0.916256305548883, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  1461 |
 |      0       |        1        |  1328 |
 |      0       |        0        |  4000 |
 |      1       |        1        | 26515 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [18]:
sent_mod.show(view = 'Evaluation')

In [19]:
# apply learned model to giraffe
giraffe_reviews['pred_sent'] = sent_mod.predict(giraffe_reviews, output_type='probability')

In [21]:
#sorting by sent
giraffe_reviews = giraffe_reviews.sort('pred_sent', ascending = False)

In [22]:
giraffe_reviews.head(2)

Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	pred_sent	float

Rows: 2

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Vulli Sophie the Giraffe T... | Sophie, oh Sophie, your ti... |  5.0   |
| Vulli Sophie the Giraffe T... | I'm not sure why Sophie is... |  4.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+----------------+
|           word_count          |   pred_sent    |
+-------------------------------+----------------+
| {'giggles': 1L, 'all': 1L,... |      1.0       |
| {'peace': 1L, 'month': 1L,... | 0.999999999703 |
+-------------------------------+----------------+
[2 rows x 5 columns]

In [23]:
print giraffe_reviews['review'][0]
print ""
print giraffe_reviews['review'][1]

Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viole

In [24]:
# show negative
print giraffe_reviews[-1]['review']
print ""
print giraffe_reviews[-2]['review']

My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!

This children's toy is nostalgic and very cute. However, there is a distinct rubber smell and a very odd taste, yes I tried it, that my baby did not enjoy. Also, if it is soiled it is extremely difficult to clean as the rubber is a kind of porus material and does not clean well. The final thing is the squeaking 

## Assignment Start

In [25]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [42]:
def keywords_count(word_count,w):  
    if w in word_count:        
        return word_count[w]    
    else :        
        return 0    

In [43]:
p = products

In [44]:
for word in selected_words: 
     products[word] = products['word_count'].apply(lambda x : keywords_count(x, word))

In [89]:
for word in selected_words:
    print word
    print products[word].sum()

awesome
2002
great
42420.0
fantastic
873.0
amazing
1305.0
love
40277.0
horrible
659
bad
3197
terrible
673.0
awful
345
wow
131
hate
1057


In [49]:
train_data,test_data = products.random_split(.8, seed=0)

In [50]:
features=selected_words

In [52]:
selected_words_model = graphlab.logistic_classifier.create(train_data,
                                              target = 'sentiment',
                                              features = selected_words,
                                              validation_set = test_data)

PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 133448
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Number of coefficients    : 12
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | 1         | 2        | 0.108103     | 0.844299          | 0.842842            |
PROGRESS: | 2         | 3        | 0.173167     | 0.844186          | 0.842842            |
PROGRESS: | 3         | 4        | 0.239231     | 0.844276          | 0.843142            |
PROGRESS: | 4         | 5        |

In [56]:
selected_words_model['coefficients']

Columns:
	name	str
	index	str
	class	int
	value	float

Rows: 12

Data:
+-------------+-------+-------+-----------------+
|     name    | index | class |      value      |
+-------------+-------+-------+-----------------+
| (intercept) |  None |   1   |  1.36728315229  |
|   awesome   |  None |   1   |  1.05800888878  |
|    great    |  None |   1   |  0.883937894898 |
|  fantastic  |  None |   1   |  0.891303090304 |
|   amazing   |  None |   1   |  0.892802422508 |
|     love    |  None |   1   |  1.39989834302  |
|   horrible  |  None |   1   |  -1.99651800559 |
|     bad     |  None |   1   | -0.985827369929 |
|   terrible  |  None |   1   |  -2.09049998487 |
|    awful    |  None |   1   |  -1.76469955631 |
+-------------+-------+-------+-----------------+
[12 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.

In [57]:
selected_words_model.evaluate(test_data, metric = 'roc_curve')

{'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 1001
 
 Data:
 +------------------+-------------------+-------------------+-------+------+
 |    threshold     |        fpr        |        tpr        |   p   |  n   |
 +------------------+-------------------+-------------------+-------+------+
 |       0.0        | 0.000188253012048 | 3.56913412806e-05 | 28018 | 5312 |
 | 0.0010000000475  |   0.999811746988  |   0.999964308659  | 28018 | 5312 |
 | 0.00200000009499 |   0.999623493976  |   0.999964308659  | 28018 | 5312 |
 | 0.00300000002608 |   0.999623493976  |   0.999964308659  | 28018 | 5312 |
 | 0.00400000018999 |   0.999435240964  |   0.999928617317  | 28018 | 5312 |
 | 0.00499999988824 |   0.999435240964  |   0.999928617317  | 28018 | 5312 |
 | 0.00600000005215 |   0.999435240964  |   0.999892925976  | 28018 | 5312 |
 | 0.00700000021607 |   0.999435240964  |   0.999892925976  | 28018 | 5312 |
 | 0.00800000037998 |   0.999435240964  |   0.999

In [90]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8431419649291376, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  234  |
 |      1       |        0        |  130  |
 |      0       |        1        |  5094 |
 |      1       |        1        | 27846 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

## Baby Trend Diaper Champ

In [58]:
diaper_champ_reviews  = products[products['name']  == 'Baby Trend Diaper Champ']

In [59]:
diaper_champ_reviews['rating'].show(view = 'Categorical')

In [92]:
64.423 + 19.799

84.22200000000001

In [61]:
diaper_champ_reviews['pred_sent'] = sent_mod.predict(diaper_champ_reviews, output_type='probability')

In [63]:
diaper_champ_reviews = diaper_champ_reviews.sort('pred_sent', ascending = False)

In [93]:
diaper_champ_reviews['pred_sent'].head()

dtype: float
Rows: 10
[0.9999999372669541, 0.9999999174063996, 0.9999998995086888, 0.9999998361817682, 0.9999998247452163, 0.9999997593154595, 0.9999996921105206, 0.9999996424880316, 0.9999996045038924, 0.9999994868042269]

In [68]:
diaper_champ_reviews['review'][0]

'Baby Luke can turn a clean diaper to a dirty diaper in 3 seconds flat. The diaper champ turns the smelly diaper into "what diaper smell" in less time than that. I hesitated and wondered what I REALLY needed for the nursery. This is one of the best purchases we made. The champ, the baby bjorn, fluerville diaper bag, and graco pack and play bassinet all vie for the best baby purchase.Great product, easy to use, economical, effective, absolutly fabulous.UpdateI knew that I loved the champ, and useing the diaper genie at a friend\'s house REALLY reinforced that!! There is no comparison, the chanp is easy and smell free, the genie was difficult to use one handed (which is absolutly vital if you have a little one on a changing pad) and there was a deffinite odor eminating from the genieplus we found that the quick tie garbage bags where the ties are integrated into the bag work really well because there isn\'t any added bulk around the sealing edge of the champ.'

In [67]:
selected_words_model.predict(diaper_champ_reviews[0:1], output_type='probability')

dtype: float
Rows: 1
[0.796940851290673]